In [4]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import os

# Set the PATH environment variable to include the directory containing chromedriver.exe
os.environ['PATH'] += r'C:/usr/chromedriver.exe'

options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
options.add_argument('user-agent={0}'.format(user_agent))

# Initialize the Chrome browser with the configuration
driver = webdriver.Chrome(options=options)

## Maximize the browser window
driver.maximize_window()

# Open the web page
driver.get('https://www.ea.com/es/games/ea-sports-fc/ratings')

time.sleep(10)

# Find the element and click on it to accept cookies
cookies = driver.find_element(By.XPATH, "//*[@id='truste-consent-required']")
cookies.click()

# Sleep to allow the cookies banner to disappear (if needed)
time.sleep(20)

# Find the table element
table = driver.find_element(By.XPATH, "/html/body/div/div[2]/div/section[1]/table/tbody")

# Get all of the image elements in the table
images = table.find_elements(By.TAG_NAME, "img")

# Iterate over the image elements and get the alt attribute of each one
alt_attributes = []
for image in images:
    alt_attribute = image.get_attribute("alt")
    alt_attributes.append(alt_attribute)

# Find the table element
table = driver.find_element(By.XPATH, "//table[@class='Table_table__C0OxY']")

headers = table.find_elements(By.TAG_NAME, "th")

header_text = [header.text for header in headers]

# Encuentra todas las filas de la tabla
rows = driver.find_elements(By.XPATH, "/html/body/div/div[2]/div/section[1]/table/tbody/tr")

# Inicializa una lista para almacenar las filas de la tabla
tablad = []

# Itera a través de las filas
for row in rows:
    # Obtiene el texto de la fila
    fila_texto = row.text

    # Divide el texto en columnas usando espacios en blanco como separador
    columnas = fila_texto.split()

    # Verifica si hay más de 3 columnas en la fila
    if len(columnas) > 3:
        # Elimina los valores de un solo dígito que aparecen después del nombre
        columnas = columnas[:3] + [x for x in columnas[3:] if not (x.isdigit() and len(x) == 1)]

    # Convierte las columnas en una cadena de texto separada por comas
    fila_csv = ",".join(columnas)

    # Agrega la fila en formato CSV a la lista
    tablad.append(fila_csv)

# Close the browser
driver.quit()



In [ ]:
import pandas as pd
# Filtrar las filas que no están vacías
mi_lista_filtrada = [elemento for elemento in tablad if elemento]

# Dividir cada elemento de la lista por comas y obtener el primer elemento y los últimos 8 elementos
datos = [elemento.split(',') for elemento in mi_lista_filtrada]
datos = [(elemento[0], *elemento[-8:]) for elemento in datos]

# Crear un DataFrame con 9 columnas
dfl = pd.DataFrame(datos, columns=[header_text[0]] + header_text[-8:])

In [ ]:
# Eliminar elementos vacíos
nueva_lista = [elemento for elemento in alt_attributes if elemento != '']

# Crear grupos de cuatro elementos cada uno
grupos = [nueva_lista[i:i+4] for i in range(0, len(nueva_lista), 4)]

# Crear un DataFrame
df = pd.DataFrame(grupos, columns=['C', 'NAC', 'EQUIPO', 'CARRERA DE'])

# Eliminar la columna 'C'
df = df.drop(columns=['C'])
df = df[['CARRERA DE', 'NAC', 'EQUIPO']]

In [25]:
result = pd.concat([df, dfl], axis=1)

result = result[header_text]
print(result)


   RANK         CARRERA DE             NAC             EQUIPO  POS GRL RIT  \
0     1      Kylian Mbappé         Francia           Paris SG   DC  91  97   
1     2    Alexia Putellas          España       FC Barcelona   MC  91  82   
2     3     Erling Haaland         Noruega    Manchester City   DC  91  89   
3     4    Kevin De Bruyne         Bélgica    Manchester City   MC  91  72   
4     5     Aitana Bonmatí          España       FC Barcelona   MC  90  81   
..  ...                ...             ...                ...  ...  ..  ..   
95   96    Ashley Lawrence          Canadá            Chelsea   LD  86  86   
96   97    Mallory Swanson  Estados Unidos  Chicago Red Stars   EI  86  93   
97   98  Wojciech Szczęsny         Polonia           Juventus  POR  86  86   
98   99  Cristiana Girelli          Italia           Juventus   DC  86  81   
99  100      Jack Grealish      Inglaterra    Manchester City   EI  85  76   

   TIR PAS REG DEF FÍS  
0   90  80  92  36  78  
1   90  91  9

In [24]:
# Guardar en un archivo CSV
result.to_csv('estadíscas_básicas_FC24.csv', index=False)